## 통계 기반 기법 개선하기

## PMI : 점별 상호 정보량(Pointwise Mutual Information), log 함수

### PPMI(Positive PMI) : 양의 상호 정보량,  PPMI(x,y) = max(0,PMI(x,y))

In [6]:
from mynlp import preprocess,create_co_matrix,cos_similarity,most_similar
import numpy as np


In [7]:
text = 'You say goodbye and I say hello.' 
corpus, word_to_id,id_to_word = preprocess(text)
print(corpus)     # 8 개
print(word_to_id) # 7 개
vocab_size = len(word_to_id)   
C = create_co_matrix(corpus,vocab_size)
print('동시발생행렬:\n',C)
C.shape  # (7,7)

[0 1 2 3 4 1 5 6]
{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}
동시발생행렬:
 [[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]


(7, 7)

In [17]:
M = np.zeros_like(C,dtype=np.float32) # (7,7)

N = np.sum(C)        # 14 , 동시발생행렬 요소전체의 합 
print(N)
S = np.sum(C,axis=0) # 동시발생행렬 요소의 수직 방향 합
print(S)  # [1 4 2 2 2 2 1]
print('동시발생행렬:\n',C)
total = C.shape[0] * C.shape[1] #  49, 동시발생행렬 요소의 총갯수

for i in range(C.shape[0]): # 7회
    for j in range(C.shape[1]) : # 7회
        # C[i,j] : 동시발생행렬의 요소한개의 값, 비교하려는 두 단어의 동시발생횟수
        # S[0] : 동시발생행렬에서 열의 'you' 단어의 동시 발생횟수
        # S[i] : 동시발생행렬에서 열의 한 단어의 동시 발생횟수 , 비교하려는 두단어 중 하나 ,x
        # S[j] : 동시발생행렬에서 열 한 단어에 대해 비교하려는 나머지 단어들의 동시 발생횟수
        # print(i,j,C[i,j])
        # print('S[%d]:'%i,S[i],'S[%d]:'%j,S[j])
        
        pmi = np.log2(C[i,j]*N /(S[i]*S[j]) + 1e-8 )
        M[i,j] = max(0,pmi) 
print('PPMI:\n',M)  # PPMI 행렬로 변환

14
[1 4 2 2 2 2 1]
동시발생행렬:
 [[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]
PPMI:
 [[0.        1.8073549 0.        0.        0.        0.        0.       ]
 [1.8073549 0.        0.8073549 0.        0.8073549 0.8073549 0.       ]
 [0.        0.8073549 0.        1.8073549 0.        0.        0.       ]
 [0.        0.        1.8073549 0.        1.8073549 0.        0.       ]
 [0.        0.8073549 0.        1.8073549 0.        0.        0.       ]
 [0.        0.8073549 0.        0.        0.        0.        2.807355 ]
 [0.        0.        0.        0.        0.        2.807355  0.       ]]


In [31]:
# PPMI 행렬 변환 함수 구현 
def ppmi(C, verbose=False, eps = 1e-8):
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0

    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / (S[i]*S[j]) + eps)
            M[i, j] = max(0, pmi)

#             if verbose:
#                 cnt += 1
#                 if cnt % (total//100) == 0:
#                     print('%.1f%% 완료' % (100*cnt/total))
    return M

In [34]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
W = ppmi(C,verbose=True)
print(W)  # 동시발생행렬보다 좀더 나은 확률에 기반한 PPMI행렬로 변환되었다

[[0.        1.8073549 0.        0.        0.        0.        0.       ]
 [1.8073549 0.        0.8073549 0.        0.8073549 0.8073549 0.       ]
 [0.        0.8073549 0.        1.8073549 0.        0.        0.       ]
 [0.        0.        1.8073549 0.        1.8073549 0.        0.       ]
 [0.        0.8073549 0.        1.8073549 0.        0.        0.       ]
 [0.        0.8073549 0.        0.        0.        0.        2.807355 ]
 [0.        0.        0.        0.        0.        2.807355  0.       ]]


### SVD(Singular Value Decomposition, 특잇값분해) 에 의한 차원의 축소

#### 차원 축소 알고리즘 정리
- PCA(Principal Component Analysis,주성분 분석) : 데이터의 분산(variance)을 최대한 보존하면서 서로 직교하는 새 기저(축)를 찾아, 고차원 공간의 표본들을 선형 연관성이 없는 저차원 공간으로 변환하는 기법, 정방행렬인 공분산행렬(covariance matrix)을 고유벡터(eigenvector)로 분해한다

 https://ratsgo.github.io/machine%20learning/2017/04/24/PCA/ 
<br>

- SVD(Singular Value Decomposition, 특잇값분해) : 임의의 행렬을 세 행렬의 곱으로 분해

$$ X=USV^T $$
      
   https://angeloyeo.github.io/2019/08/01/SVD.html
  
   https://darkpgmr.tistory.com/106 

[노트북문서화작업]
http://www.onemathematicalcat.org/MathJaxDocumentation/MathJaxKorean/TeXSyntax_ko.html